In [54]:
import os
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.modeler.advanced_cad.stackup_3d import Stackup3D

In [55]:
AEDT_VERSION = "2025.2"
NUM_CORES = 16
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

In [56]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

In [57]:
project_name = os.path.join(temp_folder.name, "patch.aedt")
hfss = ansys.aedt.core.Hfss(
    project=project_name,
    solution_type="Model",
    design="patch",
    non_graphical=NG_MODE,
    new_desktop=False,
    version=AEDT_VERSION,
)

PyAEDT INFO: Python version 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.18.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\z2538\AppData\Local\Temp\pyaedt_z2538_a389e4ea-b179-4c40-a0c6-a747a05b05f3.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 6495.
PyAEDT INFO: Electronics Desktop started on gRPC port: 6495 after 8.760507345199585 seconds.
PyAEDT INFO: AEDT installation Path D:\Program\ANSYS Inc\v252\AnsysEM
PyAEDT INFO: Ansoft.ElectronicsDesktop.2025.2 version started with process ID 33248.
PyAEDT INFO: Project patch has been created.
PyAEDT INFO: Added design 'patch' of type HFSS.
PyAEDT INFO: Aedt Objects correctly read


In [58]:
hfss.modeler.delete()

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec
PyAEDT WARNING: No objects to delete


False

## Specify units and coordinates

In [59]:
length_units = "mm"
freq_units = "GHz"
hfss.modeler.model_units = length_units

planexy = ansys.aedt.core.constants.Plane.XY
planezx = ansys.aedt.core.constants.Plane.ZX
planeyz = ansys.aedt.core.constants.Plane.YZ

hfss.modeler.set_working_coordinate_system("Global")

True

## Define constants

In [60]:
H = 1.6
L0 = 30
W0 = 37.26
L1 = 17.45
W1 = 1.16
L2 = 15
W2 = 2.98

In [61]:
substrate_origin    = [-L0, -W0, 0]
substrate_size      = [1.5*L0+L1+L2, 2*W0, H]
substrate_material  = 'FR4_epoxy'

In [62]:
air_origin    = [-L0*0.5-30, -W0*0.5-30, 0]
air_size      = [L0+L1+L2+30, W0+60, H+30]
air_mateiral  = 'vacuum'

In [63]:
tline_origin       = [0.5*L0, -0.5*W1, H]
tline_size         = [L1, W1]
tline_orientation  = planexy

patch_origin       = [-0.5*L0, -0.5*W0, H]
patch_size         = [L0, W0]
patch_orientation  = planexy

ohm50_origin       = [0.5*L0+L1, -0.5*W2, H]
ohm50_size         = [L2, W2]
ohm50_orientation  = planexy

wave_port_origin   = [0.5*L0+L1+L2, -4*W2, 0]
wave_port_size     = [8*W2, 8*H]
wave_port_orientation = planeyz

## Start

In [64]:
modeler = hfss.modeler
hfss.modeler.delete()

PyAEDT WARNING: No objects to delete


False

## Create models

In [65]:
substrate  = modeler.create_box(origin=substrate_origin, sizes=substrate_size, name='substrate', material=substrate_material)
airbox     = modeler.create_box(origin=air_origin, sizes=air_size, name='airbox', material=air_mateiral)

PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


In [66]:
tline     = modeler.create_rectangle(orientation=tline_orientation, origin=tline_origin, sizes=tline_size, name='tline')
patch     = modeler.create_rectangle(orientation=patch_orientation, origin=patch_origin, sizes=patch_size, name='patch')
ohm50     = modeler.create_rectangle(orientation=ohm50_orientation, origin=ohm50_origin, sizes=ohm50_size, name='ohm50')
wave_port = modeler.create_rectangle(orientation=wave_port_orientation, origin=wave_port_origin, sizes=wave_port_size, name='wave_port')

In [67]:
antenna = modeler.unite([tline, patch, ohm50])

PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Union of 3 objects has been executed.


## Assign boundary

In [68]:
hfss.assign_perfect_e(assignment=antenna)
hfss.wave_port(assignment=wave_port, name='wave_port')
hfss.assign_radiation_boundary_to_objects(assignment=airbox, name='air_radiation')

PyAEDT INFO: Boundary Perfect E PerfectE_tline has been created.
PyAEDT INFO: Boundary Wave Port wave_port has been created.
PyAEDT INFO: Boundary Radiation air_radiation has been created.


In [69]:
bottom_face_id = -1
min_z = 1e9

# Find the top face
for face in substrate.faces:
    face_center = face.center
    if face_center[2] < min_z:
        min_z = face_center[2]
        bottom_face_id = face.id

if bottom_face_id == -1:
    print("No top face found")
else:
    hfss.assign_perfect_e(assignment=[bottom_face_id], name='PerfectE_substrate')

PyAEDT INFO: Boundary Perfect E PerfectE_substrate has been created.


## Define setup

In [70]:
solution_name = 'Setup1'

basic = {'name': solution_name,
         'Frequency': '2.45GHz',
         'MaximumPasses': 20,
         'MaxDeltaS': 0.02}

sweep1 = {'name': 'Sweep1',
          'unit': 'GHz',
          'start_frequency': 1.5,
          'stop_frequency': 3.5,
          'step_size':0.01,
          "sweep_type": "Interpolating",
          'save_fields': True}


In [71]:
setup = hfss.create_setup(**basic)

setup.create_linear_step_sweep(**sweep1)

PyAEDT INFO: Parsing C:/Users/z2538/AppData/Local/Temp/tmp6dyss27x.ansys/patch.aedt.
PyAEDT INFO: File C:/Users/z2538/AppData/Local/Temp/tmp6dyss27x.ansys/patch.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: aedt file load time 0.043332815170288086
PyAEDT INFO: Linear step sweep Sweep1 has been correctly created


Setup1 : Sweep1

## Save and print

In [72]:
hfss.save_project()

PyAEDT INFO: Project patch Saved correctly


True

In [73]:
message = "We have created a patch antenna"
message += "using PyAEDT.\n\nThe project file is "
message += f"located at \n'{hfss.project_file}'.\n"
message += f"\nThe HFSS design is named '{hfss.design_name}'\n"
message += f"and is comprised of "
message += f"{len(hfss.modeler.objects)} objects whose names are:\n\n"
message += "".join([f"- '{o.name}'\n" for _, o in hfss.modeler.objects.items()])
print(message)

We have created a patch antennausing PyAEDT.

The project file is located at 
'C:/Users/z2538/AppData/Local/Temp/tmp6dyss27x.ansys/patch.aedt'.

The HFSS design is named 'patch'
and is comprised of 4 objects whose names are:

- 'substrate'
- 'airbox'
- 'tline'
- 'wave_port'



## Run analysis

In [74]:
hfss.analyze(cores=NUM_CORES)

PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS correctly changed.
PyAEDT INFO: Solving all design setups.
PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS correctly changed.
PyAEDT INFO: Design setup None solved correctly in 0.0h 0.0m 49.0s


True

## Postprocessing

In [75]:
post = hfss.post

s_parameter_expression = 'dB(S(wave_port,wave_port))'

report = post.create_report(
    expressions=s_parameter_expression,
    report_category="S Parameters",  
    plot_type="Rectangular Plot",
    plot_name="S11_dB_Plot", 
)

PyAEDT INFO: Parsing C:/Users/z2538/AppData/Local/Temp/tmp6dyss27x.ansys/patch.aedt.
PyAEDT INFO: File C:/Users/z2538/AppData/Local/Temp/tmp6dyss27x.ansys/patch.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: aedt file load time 0.04479622840881348
PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


In [52]:
hfss.release_desktop()
time.sleep(3)

PyAEDT INFO: Desktop has been released and closed.


In [53]:
temp_folder.cleanup()

## Export settings

In [176]:
import pyaedt

def dump_aedt_settings_final(hfss_app, output_file_path):
    """
    连接到一个HFSS App，并将其几何、边界、求解等所有关键配置导出到文本文件。
    """
    try:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write("="*60 + "\n")
            f.write(f"PROJECT CONFIGURATION DUMP FOR: {hfss_app.project_name}\n")
            f.write(f"DESIGN: {hfss_app.design_name}\n")
            f.write("="*60 + "\n\n")

            # --- 1. 导出详细的几何信息 (Geometry Details) ---
            f.write("-" * 25 + " Geometry Details " + "-" * 25 + "\n")
            if not hfss_app.modeler.objects:
                f.write("No geometry objects found.\n")
            else:
                # 按名称排序，确保两个文件中的顺序一致，方便对比
                sorted_objects = sorted(hfss_app.modeler.objects.values(), key=lambda o: o.name)
                f.write(f"Total Objects: {len(sorted_objects)}\n\n")
                for obj in sorted_objects:
                    f.write(f"Object Name: {obj.name}\n")
                    f.write(f"  - Type: {obj.object_type}\n")
                    f.write(f"  - Material: {obj.material_name}\n")
                    # 边界框是最关键的几何信息，包含了位置和尺寸
                    # 格式: [Xmin, Ymin, Zmin, Xmax, Ymax, Zmax]
                    f.write(f"  - Bounding Box: {obj.bounding_box}\n")
                    f.write(f"  - Vertices: {len(obj.vertices)}, Edges: {len(obj.edges)}, Faces: {len(obj.faces)}\n\n")

            # --- 2. 导出边界条件 (Boundaries) ---
            f.write("-" * 25 + " Boundaries " + "-" * 25 + "\n")
            if not hfss_app.boundaries:
                f.write("No boundaries found.\n")
            else:
                sorted_boundaries = sorted(hfss_app.boundaries, key=lambda b: b.name)
                for boundary in sorted_boundaries:
                    f.write(f"Boundary Name: {boundary.name}\n")
                    f.write(f"  - Type: {boundary.type}\n")
                    assignment = boundary.props.get('Objects') or boundary.props.get('Faces') or boundary.props.get('Edges')
                    f.write(f"  - Assigned To: {assignment}\n\n")

            # --- 3. 导出激励 (Excitations) ---
            f.write("-" * 25 + " Excitations " + "-" * 25 + "\n")
            if not hfss_app.excitations:
                f.write("No excitations found.\n")
            else:
                 sorted_excitations = sorted(hfss_app.excitations)
                 for exc_name in sorted_excitations:
                    f.write(f"Excitation Name: {exc_name}\n")
            f.write("\n")
            
            # --- 4. 导出分析设置 (Analysis Setup) ---
            f.write("-" * 25 + " Analysis Setups " + "-" * 25 + "\n")
            # (这部分代码和之前一样，为了完整性而保留)
            if not hfss_app.setups:
                f.write("No setups found.\n")
            else:
                sorted_setups = sorted(hfss_app.setups, key=lambda s: s.name)
                for setup in sorted_setups:
                    f.write(f"Setup Name: {setup.name}\n")
                    sorted_props = sorted(setup.props.items())
                    for key, value in sorted_props:
                        f.write(f"  - {key}: {value}\n")
                    f.write("\n")
        
        print(f"包含几何信息的配置已成功导出到: {output_file_path}")
        
    except Exception as e:
        print(f"导出配置时发生错误: {e}")

In [177]:
# --- 使用新的“超级导出”函数来对比两个工程 ---

# 准备工作：在AEDT中，手动打开【手动创建】的、结果正确的工程。

# 1. 导出“手动工程”的配置
print(">>> 步骤1：正在导出 手动工程 的详细配置...")
hfss_manual_app = pyaedt.Hfss(new_desktop=False) 
dump_aedt_settings_final(hfss_manual_app, "manual_settings_detailed.txt")
print(">>> 'manual_settings_detailed.txt' 已生成。")


# 准备工作2：在AEDT中，切换到【脚本生成的】、结果错误的工程。

# 2. 导出“脚本工程”的配置
print("\n>>> 步骤2：请在AEDT中切换到【脚本生成的工程】，然后在这里按Enter键继续...")
input() 

print(">>> 正在导出 脚本工程 的详细配置...")
hfss_script_app = pyaedt.Hfss(new_desktop=False)
dump_aedt_settings_final(hfss_script_app, "script_settings_detailed.txt")
print(">>> 'script_settings_detailed.txt' 已生成。")


# 3. 进行最终对比
print("\n>>> 步骤3：导出完成！")
print("请使用文本对比工具比较 'manual_settings_detailed.txt' 和 'script_settings_detailed.txt'。")

>>> 步骤1：正在导出 手动工程 的详细配置...
PyAEDT INFO: Python version 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.18.0.
PyAEDT INFO: Returning found Desktop session with PID 44684!
PyAEDT INFO: No project is defined. Project patch1 exists and has been read.
PyAEDT INFO: Active Design set to HFSSDesign1
PyAEDT INFO: Active Design set to HFSSDesign1
PyAEDT INFO: Aedt Objects correctly read
PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Parsing E:/HFSS/projects/patch1.aedt.
PyAEDT INFO: File E:/HFSS/projects/patch1.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: aedt file load time 0.04542040824890137
包含几何信息的配置已成功导出到: manual_settings_detailed.txt